In [1]:
from __future__ import print_function, division
import pylab as pl
import pandas as pd
import numpy as np

In [2]:
!cd /nfshome/fb1182/PUIdata/

In [3]:
!ls

201601-citibike-tripdata.csv  Assignment1.ipynb		  HW3_fb1182
201601-citibike-tripdata.zip  FHVBaseAggregateReport.csv  Untitled.ipynb


In [4]:
!rm -rf 201601-citibike-tripdata.csv

In [5]:
!curl -O https://s3.amazonaws.com/tripdata/201601-citibike-tripdata.zip 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.9M  100 16.9M    0     0  26.3M      0 --:--:-- --:--:-- --:--:-- 26.3M


In [6]:
!unzip 201601-citibike-tripdata.zip 

Archive:  201601-citibike-tripdata.zip
  inflating: 201601-citibike-tripdata.csv  


In [7]:
!ls

201601-citibike-tripdata.csv  Assignment1.ipynb		  HW3_fb1182
201601-citibike-tripdata.zip  FHVBaseAggregateReport.csv  Untitled.ipynb


IDEA:
Children of ages (16,18) are more likely to choose high schools around End Station Name A than high schools near other End Station Names

NULL HYPOTHESIS:
    
The proportion of children of ages (16,18) docking their bikes during the week around 8:00 a.m that is between(7:55 a.m - 8:30 a.m) at other End Station Names is the same or lower than the
proportion of children(16,18) docking their bikes at End Station Name A around the same time, during the week. 

_$H_0$ : $\frac{Children(16,18){\mathrm{Other End Station Names}}}{Children(16,18){\mathrm{total}}} <= \frac{Children(16,18){\mathrm{End Station Name A}}}{Children(16,18)_{\mathrm{total}}}$
    
_$H_1$ : $\frac{Children(16,18){\mathrm{Other End Station Names}}}{Children(16,18){\mathrm{total}}} > \frac{Children(16,18){\mathrm{End Station Name A}}}{Children(16,18)_{\mathrm{total}}}$

or identically:

_$H_0$ : $\frac{Children(16,18){\mathrm{Other End Station Names}}}{Children(16,18){\mathrm{total}}} - \frac{Children(16,18){\mathrm{End Station Name A}}}{Children(16,18)_{\mathrm{Total}}} <= 0
    
_$H_1$ : $\frac{Children(16,18){\mathrm{Other End Station Names}}}{Children(16,18){\mathrm{total}}} - \frac{Children(16,18){\mathrm{End Station Name A}}}{Children(16,18)_{\mathrm{Total}}} > 0

We will use a significance level  $\alpha=0.05$

Which means we want the probability of getting a result at least as significant as ours to be less then 5%

For this test we are starting with a single month of data: reading data from citibike csv file from Jan 2016
    
What is wanted is the date - to determine whether or not the trip occurred during the week as opposed to the weekend, the time - to determine the trip is taking place around 8:00 a.m, birth date of subscriber to check whether rider is between given age, and End Station for each trip. Preserving stoptime; endstation name; birthyear; date. 

In [8]:
df = pd.read_csv('201601-citibike-tripdata.csv')
df['date'] = pd.to_datetime(df['starttime'])

In [9]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date
0,923,1/1/2016 00:00:41,1/1/2016 00:16:04,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22285,Subscriber,1958.0,1,2016-01-01 00:00:41
1,379,1/1/2016 00:00:45,1/1/2016 00:07:04,476,E 31 St & 3 Ave,40.743943,-73.979661,498,Broadway & W 32 St,40.748549,-73.988084,17827,Subscriber,1969.0,1,2016-01-01 00:00:45
2,589,1/1/2016 00:00:48,1/1/2016 00:10:37,489,10 Ave & W 28 St,40.750664,-74.001768,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,21997,Subscriber,1982.0,2,2016-01-01 00:00:48
3,889,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961.0,2,2016-01-01 00:01:06
4,1480,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952.0,1,2016-01-01 00:01:12


In [10]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'date'],
      dtype='object')

In [11]:
df.drop(['tripduration', 'starttime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'gender'], axis=1, inplace=True)

In [12]:
df.head()

,stoptime,end station name,birth year,date
0,1/1/2016 00:16:04,South End Ave & Liberty St,1958.0,2016-01-01 00:00:41
1,1/1/2016 00:07:04,Broadway & W 32 St,1969.0,2016-01-01 00:00:45
2,1/1/2016 00:10:37,Greenwich Ave & 8 Ave,1982.0,2016-01-01 00:00:48
3,1/1/2016 00:15:56,South End Ave & Liberty St,1961.0,2016-01-01 00:01:06
4,1/1/2016 00:25:52,Central Park S & 6 Ave,1952.0,2016-01-01 00:01:12


In [19]:
df['birth year'] = df['birth year'].astype(int).astype(str)

In [20]:
df.head()

,stoptime,end station name,birth year,date
0,1/1/2016 00:16:04,South End Ave & Liberty St,1958,2016-01-01 00:00:41
1,1/1/2016 00:07:04,Broadway & W 32 St,1969,2016-01-01 00:00:45
2,1/1/2016 00:10:37,Greenwich Ave & 8 Ave,1982,2016-01-01 00:00:48
3,1/1/2016 00:15:56,South End Ave & Liberty St,1961,2016-01-01 00:01:06
4,1/1/2016 00:25:52,Central Park S & 6 Ave,1952,2016-01-01 00:01:12


In [13]:
df1 = df[df.isnull().any(axis=1)]

In [14]:
print(df1)

                  stoptime             end station name  birth year  \
19       1/1/2016 00:23:49      Allen St & Rivington St         NaN   
20       1/1/2016 00:23:54      Allen St & Rivington St         NaN   
22       1/1/2016 00:12:15               E 4 St & 2 Ave         NaN   
26       1/1/2016 00:44:58              9 Ave & W 45 St         NaN   
28       1/1/2016 00:32:50           E 67 St & Park Ave         NaN   
30       1/1/2016 00:32:01              E 11 St & 2 Ave         NaN   
31       1/1/2016 00:32:03              E 11 St & 2 Ave         NaN   
32       1/1/2016 00:40:51              E 55 St & 2 Ave         NaN   
52       1/1/2016 00:42:08              W 27 St & 7 Ave         NaN   
53       1/1/2016 00:42:10              W 27 St & 7 Ave         NaN   
73       1/1/2016 00:43:26              8 Ave & W 33 St         NaN   
74       1/1/2016 00:43:24              8 Ave & W 33 St         NaN   
76       1/1/2016 00:43:15              8 Ave & W 33 St         NaN   
80    

In [15]:
print (df.isnull())

       stoptime end station name birth year   date
0         False            False      False  False
1         False            False      False  False
2         False            False      False  False
3         False            False      False  False
4         False            False      False  False
5         False            False      False  False
6         False            False      False  False
7         False            False      False  False
8         False            False      False  False
9         False            False      False  False
10        False            False      False  False
11        False            False      False  False
12        False            False      False  False
13        False            False      False  False
14        False            False      False  False
15        False            False      False  False
16        False            False      False  False
17        False            False      False  False
18        False            Fals

In [16]:
df = df[np.isfinite(df['birth year'])]

In [17]:
print (df.isnull())

       stoptime end station name birth year   date
0         False            False      False  False
1         False            False      False  False
2         False            False      False  False
3         False            False      False  False
4         False            False      False  False
5         False            False      False  False
6         False            False      False  False
7         False            False      False  False
8         False            False      False  False
9         False            False      False  False
10        False            False      False  False
11        False            False      False  False
12        False            False      False  False
13        False            False      False  False
14        False            False      False  False
15        False            False      False  False
16        False            False      False  False
17        False            False      False  False
18        False            Fals